In [1]:
#importing the relevant libraries
import pandas as pd
import numpy as np
import os
from glob import glob
import sys
pd.set_option('display.max_columns', None, 'display.max_rows', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# pd.set_option('max_colwidth',-1)

In [53]:
#Collecting and concatenating all csv files per year
path_folder = "./Total_Lap_Num/"
folder_list = os.listdir("./Total_Lap_Num/")
EXT = "*.csv"
for sub in folder_list:
    PATH = path_folder + sub
    all_csv_files = [file 
                     for path, subdir, files in os.walk(PATH)
                     for file in glob(os.path.join(path,EXT))
                    ]
    df = pd.concat((pd.read_csv(f, sep=";", encoding = "ISO-8859-1").assign(file_loc=f) for f in all_csv_files))

    df = df.rename(columns=lambda x: x.strip())
    df.to_csv('./cleaned1/' + sub + '.csv', index = False)

In [54]:
#Concatenating datasets cross the years
data_14 = pd.read_csv('./cleaned1/2014-2015.CSV')
data_15 = pd.read_csv('./cleaned1/2015-2016.CSV')
data_16 = pd.read_csv('./cleaned1/2016-2017.CSV')
data_17 = pd.read_csv('./cleaned1/2017-2018.CSV')
data_18 = pd.read_csv('./cleaned1/2018-2019.CSV')

col14 = data_14.columns
col15 = data_15.columns
col16 = data_16.columns
col17 = data_17.columns
col18 = data_18.columns

all_cols = list(set(col14) | set(col15) | set(col16) |set(col17) |set(col18))
common_cols = list(set(col14) & set(col15) & set(col16) & set(col17) & set(col18))

In [70]:
all_contest_data = ['./cleaned1/2014-2015.CSV','./cleaned1/2015-2016.CSV','./cleaned1/2016-2017.CSV',
                    './cleaned1/2017-2018.CSV','./cleaned1/2018-2019.CSV']
indi_pd = []
cols_wit_str = ['TEAM','DRIVER_NAME','CROSSING_FINISH_LINE_IN_PIT']
cols_wit_num = ['LAPS', 'GROUP']
cols_to_norm = ['KPH', 'S1', 'S2', 'S3', 'PIT_TIME', 'ELAPSED']
for f in all_contest_data:
    temp = pd.read_csv(f)[common_cols]
    temp = temp.drop(columns = ['DRIVER_SECONDNAME', 'STATUS', 'DRIVER_FIRSTNAME',
       'TOP_SPEED', 'GAP_FIRST', 'ï»¿NUMBER', 'S1_IMPROVEMENT',
       'MANUFACTURER', 'NUMBER', 'HOUR', 'FL_LAPNUM', 'ï»¿POSITION',
       'GAP_PREVIOUS', 'FL_KPH', 'DIVISION', 'DRIVER_LICENSE', 'S1_LARGE',
       'DRIVER_HOMETOWN', 'DRIVER_SHORTNAME', 'VEHICLE',
       'CLASS', 'S3_LARGE', 'S2_IMPROVEMENT', 'TIRES', 'S3_IMPROVEMENT',
       'DRIVER_COUNTRY', 'S2_LARGE', 'FL_TIME',
       'LAP_IMPROVEMENT', 'LAP_TIME', 'LAP_NUMBER', 'Unnamed: 25', 'DRIVER_NUMBER',
       'TOTAL_TIME'])
    for i in (['S1', 'S2', 'S3', 'PIT_TIME', 'ELAPSED']):
        p = temp[i]
        for count, j in enumerate(p):
            x = len(str(j).split(':'))
            if (pd.isna(j)):
                p[count] = "00:00:00"
            elif (x == 1):
                p[count] = "00:00:" + str(j)
            elif (x == 2):
                p[count] = "00:" + str(j)
            elif (x == 3):
                p[count] = "" + str(j)
            p[count] = float(p[count].split(":")[0])*3600000 + float(p[count].split(":")[1])*60000 + float(p[count].split(":")[2])*1000
        temp[i] = p
    temp[cols_to_norm] = temp[cols_to_norm].fillna(0)
    temp[cols_wit_num] = temp[cols_wit_num].fillna(0)
    temp[cols_wit_str] = temp[cols_wit_str].fillna('')
    temp[cols_to_norm] = temp[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    indi_pd.append(temp)

C:\Users\Sai\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Sai\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Sai\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Sai\AppData\Roaming\Python\Python37\site-pa

In [71]:
df = pd.concat(indi_pd)
df.head(1)

,KPH,LAPS,TEAM,PIT_TIME,DRIVER_NAME,S2,file_loc,GROUP,ELAPSED,CROSSING_FINISH_LINE_IN_PIT,S1,S3
0,0.037599,0.0,,1.0,Jarno Trulli,0.363217,./Total_Lap_Num/2014-2015\Bejing\Race\Analysis...,0,0.731826,,1.0,0.071061


In [72]:
df = df.assign(season_year = lambda x: x['file_loc'].str.split('\\').str[0])\
            .assign(season_year = lambda x: x['season_year'].str.split('/').str[2])\
                .assign(location = lambda x: x['file_loc'].str.split('\\').str[1])\
                    .drop(columns = ['file_loc'])
df.head(5)

,KPH,LAPS,TEAM,PIT_TIME,DRIVER_NAME,S2,GROUP,ELAPSED,CROSSING_FINISH_LINE_IN_PIT,S1,S3,season_year,location
0,0.037599,0.0,,1.0,Jarno Trulli,0.363217,0,0.731826,,1.000000,0.071061,2014-2015,Bejing
1,0.798265,0.0,,0.0,Jarno Trulli,0.345924,0,0.766592,,0.017009,0.073091,2014-2015,Bejing
2,0.789588,0.0,,0.0,Lucas di Grassi,0.349779,0,0.035141,,0.018427,0.068375,2014-2015,Bejing
3,0.509761,0.0,,0.0,Lucas di Grassi,0.707102,0,0.089547,,0.017363,0.122310,2014-2015,Bejing
4,0.326826,0.0,,0.0,Lucas di Grassi,0.932772,0,0.174499,,0.040429,0.166113,2014-2015,Bejing


In [117]:
aggregation = {
            'KPH':[('mean_KPH','mean'),
                ('std_KPH','std'),
                ('med_KPH','median'),
                ('max_KPH','max'),
                ('min_KPH','min'),
                ('range_KPH',lambda x: max(x) - min(x)),
                ('skew_KPH','skew')],
              'S1' : [('mean_S1','mean'),
                ('std_S1','std'),
                ('med_S1','median'),
                ('max_S1','max'),
                ('min_S1','min'),
                ('range_S1',lambda x: max(x) - min(x)),
                ('skew_S1','skew')],
              'S2' : [('mean_S2','mean'),
                ('std_S2','std'),
                ('med_S2','median'),
                ('max_S2','max'),
                ('min_S2','min'),
                ('range_S2',lambda x: max(x) - min(x)),
                ('skew_S2','skew')],
              'S3' : [('mean_S3','mean'),
                ('std_S3','std'),
                ('med_S3','median'),
                ('max_S3','max'),
                ('min_S3','min'),
                ('range_S3',lambda x: max(x) - min(x)),
                ('skew_S3','skew')],
               'LAPS': [('Total_Lap_Num','count')],
               'PIT_TIME': [('PIT_TIME_sum','sum')],
               'CROSSING_FINISH_LINE_IN_PIT': [('cnt_CROSSING_FINISH_LINE_IN_PIT',lambda x: len(x.sum()))],
              }

df['match_key'] = df['DRIVER_NAME'] + df['season_year'] + df['location']
df1 = df.groupby(['match_key','DRIVER_NAME','location','season_year','GROUP','TEAM',], as_index=True).agg(aggregation).reset_index()
df1.columns = df1.columns.droplevel()
column_names = df1.columns.values
column_names[:6] = ['match_key','DRIVER_NAME','location','season_year','GROUP', 'TEAM']
df1 = df1.drop(columns = ['match_key'])
df1['GROUP'] = df1['GROUP'].replace({'A':0,'B':0})
df1 = df1[df1['DRIVER_NAME'] != '']
df1 = df1.fillna(0)
df1.to_csv('./cleaned1/race_final_processed.csv',index = False)
df1.head(5)

,DRIVER_NAME,location,season_year,GROUP,TEAM,mean_KPH,std_KPH,med_KPH,max_KPH,min_KPH,range_KPH,skew_KPH,mean_S1,std_S1,med_S1,max_S1,min_S1,range_S1,skew_S1,mean_S2,std_S2,med_S2,max_S2,min_S2,range_S2,skew_S2,mean_S3,std_S3,med_S3,max_S3,min_S3,range_S3,skew_S3,Total_Lap_Num,PIT_TIME_sum,cnt_CROSSING_FINISH_LINE_IN_PIT
0,MA Qing Hua,Buenos,2016-2017,0,TECHEETAH,0.643436,0.068480,0.660976,0.755014,0.372358,0.382656,-3.401031,0.020251,0.011196,0.018146,0.071552,0.000000,0.071552,3.633619,0.166375,0.031363,0.167355,0.243448,0.000000,0.243448,-4.056665,0.019508,0.001400,0.019059,0.026749,0.018743,0.008007,4.284716,36,0.044202,1
1,MA Qing Hua,HongKong,2016-2017,2,TECHEETAH,0.104065,0.000000,0.104065,0.104065,0.104065,0.000000,0.000000,0.024041,0.000000,0.024041,0.024041,0.024041,0.000000,0.000000,0.467256,0.000000,0.467256,0.467256,0.467256,0.000000,0.000000,0.166735,0.000000,0.166735,0.166735,0.166735,0.000000,0.000000,1,0.000000,1
2,MA Qing Hua,Marrakesh,2016-2017,0,TECHEETAH,0.661197,0.043820,0.672087,0.678591,0.434146,0.244444,-4.741612,0.030492,0.007837,0.028956,0.073773,0.028719,0.045054,5.599985,0.253147,0.006555,0.250969,0.280617,0.247322,0.033296,2.893737,0.012066,0.000408,0.011988,0.014143,0.011724,0.002418,4.462909,33,0.035236,1
3,MA Qing Hua,Newyork2,2017-2018,0,NIO Formula E Team,0.784247,0.124418,0.835249,0.849042,0.393870,0.455172,-2.502651,0.041836,0.012217,0.039221,0.075838,0.000000,0.075838,0.896637,0.243905,0.080328,0.228327,0.503911,0.000000,0.503911,1.833349,0.017313,0.006112,0.015107,0.040007,0.014829,0.025178,2.886137,43,0.025305,1
4,MA Qing Hua,Paris,2017-2018,0,NIO Formula E Team,0.780729,0.104679,0.813027,0.823755,0.213027,0.610728,-4.546103,0.027873,0.020441,0.024516,0.164834,0.024182,0.140652,6.825230,0.242827,0.024649,0.235317,0.398867,0.231570,0.167297,5.758739,0.014480,0.005589,0.013309,0.050895,0.013071,0.037825,6.318397,47,0.035129,1


In [6]:
data_wheather = pd.read_csv('./cleaned1/wheather.CSV',decimal=",")
data_wheather['DATE_ONLY'] = pd.to_datetime(data_wheather['TIME_UTC_STR']).dt.strftime('%m/%d/%Y')
data_wheather = data_wheather.sort_values(by=['DATE_ONLY']).drop(columns=["Unnamed: 8"])

data_wheather = data_wheather.assign(season_year = lambda x: x['file_loc'].str.split('\\').str[1])\
                .assign(location = lambda x: x['file_loc'].str.split('\\').str[2])\
                .assign(match_type = lambda x: x['file_loc'].str.split('\\').str[3])\
                .assign(match_name = lambda x: x['file_loc'].str.split('\\').str[4])\
                .assign(match_name = lambda x: x['match_name'].str.split('_').str[2])\
                .assign(match_name = lambda x: x['match_name'].str.split('.').str[0])\
                .drop(columns = ['file_loc'])

In [8]:
aggregation = {'AIR_TEMP':[('mean_air_temp','mean'),
                ('std_air_temp','std'),
                ('med_air_temp','median'),
                ('max_air_temp','max'),
                ('min_air_temp','min'),
                ('range_air_temp',lambda x: max(x) - min(x)),
                ('skew_air_temp','skew')],
              'TRACK_TEMP' : [('mean_track_temp','mean'),
                ('std_track_temp','std'),
                ('med_track_temp','median'),
                ('max_track_temp','max'),
                ('min_track_temp','min'),
                ('range_track_temp',lambda x: max(x) - min(x)),
                ('skew_track_temp','skew')],
                'HUMIDITY' : [('mean_HUMIDITY','mean'),
                            ('std_HUMIDITY','std'),
                            ('med_HUMIDITY','median'),
                            ('max_HUMIDITY','max'),
                            ('min_HUMIDITY','min'),
                            ('range_HUMIDITY',lambda x: max(x) - min(x)),
                            ('skew_HUMIDITY','skew')],
                'PRESSURE' : [('mean_PRESSURE','mean'),
                    ('std_PRESSURE','std'),
                    ('med_PRESSURE','median'),
                    ('max_PRESSURE','max'),
                    ('min_PRESSURE','min'),
                    ('range_PRESSURE',lambda x: max(x) - min(x)),
                    ('skew_PRESSURE','skew')],

            'WIND_SPEED' : [('mean_WIND_SPEED','mean'),
                          ('std_WIND_SPEED','std'),
                          ('med_WIND_SPEED','median'),
                          ('max_WIND_SPEED','max'),
                          ('min_WIND_SPEED','min'),
                          ('range_WIND_SPEED',lambda x: max(x) - min(x)),
                          ('skew_WIND_SPEED','skew'),
                        ]
              }

# data_wheather[aggregation.keys] = data_wheather[aggregation.keys()].apply(pd.to_numeric)

data_wheather['match_key'] = data_wheather['DATE_ONLY'] + data_wheather['match_name']
df = data_wheather.groupby(['match_key','DATE_ONLY','location','season_year','match_name','match_type'], as_index=True).agg(aggregation).reset_index()
df.columns = df.columns.droplevel()
column_names = df.columns.values
column_names[:6] = ['match_key','DATE_ONLY','location','season_year','match_name','match_type']
df.columns = column_names
df = df.drop(columns = ['match_key','DATE_ONLY'])
df.to_csv('./cleaned1/weather_processed.csv',index = False)

In [2]:
dr = pd.read_csv('./cleaned/race_final_processed.csv')
dw = pd.read_csv('./cleaned/weather_processed.csv')
final_data = dr.merge(dw, how = 'left', on = ['location','season_year','match_name','match_type'])
final_data_with_weather = final_data[final_data['mean_air_temp'].notnull()]
final_data_with_weather.shape, dr.shape

KeyError: 'match_name'

In [87]:
from sklearn.preprocessing import OneHotEncoder

def encode_column_to_dataframe(df, allcols):
    '''Takes the column name as a string and adds to the dataframe'''
    for col in allcols:
        #dum_df = pd.get_dummies(df, columns=col, prefix=col)
        # creating instance of one-hot-encoder
        enc = OneHotEncoder(handle_unknown='ignore')
        # passing bridge-types-cat column (label encoded values of bridge_types)
        enc_df = pd.DataFrame(enc.fit_transform(df[[col]]).toarray())
        # merge with main df bridge_df on key values
        df = df.join(enc_df.add_prefix(col+"_"))
    return df

In [88]:
def get_encoded_dataframe(final):
    final_data = encode_column_to_dataframe(final,['DRIVER_NAME','location','season_year','TEAM'])
    final_data.drop(columns = ['DRIVER_NAME','location','season_year', 'TEAM'],inplace = True)
    #final_copy.to_csv('./cleaned/final_processed_before_ml.csv')
    #drop only if that column exists originally
    #final_data.drop(columns = ['Unnamed: 0'],inplace = True)
    return final_data

#### *Without Weather Model Running*

In [140]:
#Generating model-ready data using one hot encoding

final_df = get_encoded_dataframe(df1)
final_df = final_df.fillna(0)
y = final_df['Total_Lap_Num'].to_numpy()
X = final_df.drop(columns = ['Total_Lap_Num']).to_numpy()

In [141]:
from sklearn import ensemble, linear_model, neighbors, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [149]:
def test_model(X, y, model_name, model):
    print("MODEL: {}".format(model_name))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model.fit(X_train, y_train)
    Z = model.predict(X_test)
    print("R2 Value: ", r2_score(Z, y_test))
    print("MSE Value: ", mean_squared_error(Z, y_test))
    
models = {
#     "RANSACRegressor": linear_model.RANSACRegressor(random_state=0),
#     "ElasticNet": linear_model.ElasticNet(random_state=0),
    "Random Forest": ensemble.RandomForestRegressor(n_estimators=100000,
                                                   bootstrap = True,
                                                   ),
#     "K-Nearest Neighbors": neighbors.KNeighborsRegressor(),
#     "SVR": svm.SVR(),
#     "BayesianRidge": linear_model.BayesianRidge(),
    "Linear Regression": linear_model.LinearRegression()
}

In [150]:
for model in models.keys():
    print('----------------------\n')
    test_model(X,y,model, models[model])
    

----------------------

MODEL: Random Forest
R2 Value:  0.9163640495219427
MSE Value:  11.72081607412293
----------------------

MODEL: Linear Regression
R2 Value:  0.6630807601641637
MSE Value:  40.12312704707408


NameError: name 'Z' is not defined

#### *With Weather Model Running*

In [19]:
final_data_with_weather.head()

,DRIVER_NAME,location,season_year,match_name,match_type,GROUP,TEAM,mean_KPH,std_KPH,med_KPH,max_KPH,min_KPH,range_KPH,skew_KPH,mean_S1,std_S1,med_S1,max_S1,min_S1,range_S1,skew_S1,mean_S2,std_S2,med_S2,max_S2,min_S2,range_S2,skew_S2,mean_S3,std_S3,med_S3,max_S3,min_S3,range_S3,skew_S3,Total_Lap_Num,PIT_TIME_cnt,cnt_CROSSING_FINISH_LINE_IN_PIT,DATE_ONLY,mean_air_temp,std_air_temp,med_air_temp,max_air_temp,min_air_temp,range_air_temp,skew_air_temp,mean_track_temp,std_track_temp,med_track_temp,max_track_temp,min_track_temp,range_track_temp,skew_track_temp,mean_HUMIDITY,std_HUMIDITY,med_HUMIDITY,max_HUMIDITY,min_HUMIDITY,range_HUMIDITY,skew_HUMIDITY,mean_PRESSURE,std_PRESSURE,med_PRESSURE,max_PRESSURE,min_PRESSURE,range_PRESSURE,skew_PRESSURE,mean_WIND_SPEED,std_WIND_SPEED,med_WIND_SPEED,max_WIND_SPEED,min_WIND_SPEED,range_WIND_SPEED,skew_WIND_SPEED
0,MA Qing Hua,Buenos,2016-2017,Groups Qualifying Session,Q,4.0,TECHEETAH,55.950000,74.034080,55.95,108.3,3.6,104.7,0.000000,1.205626e+06,1.673677e+06,1205626.5,2389095.0,22158.0,2366937.0,0.000000,28569.500000,2348.301620,28569.5,30230.0,26909.0,3321.0,0.000000,33244.500000,132.228968,33244.5,33338.0,33151.0,187.0,0.000000,2.0,2,0,02/18/2017,26.299227,0.067998,26.2778,26.4444,26.2222,0.2222,0.634238,26.250000,0.243363,26.1111,26.6667,26.1111,0.5556,1.195861,74.931818,0.899554,75.0,76.0,73.0,3.0,-0.665006,1008.236136,0.126201,1008.230,1008.470,1008.030,0.440,0.053279,4.462272,2.049350,4.828030,8.04672,0.00000,8.04672,0.095071
1,MA Qing Hua,HongKong,2016-2017,Groups Qualifying Session,Q,2.0,TECHEETAH,60.720000,44.706677,83.70,103.4,9.6,93.8,-0.504993,1.454818e+05,2.776263e+05,21419.0,642077.0,18214.0,623863.0,2.235218,25340.400000,3472.268898,25310.0,30446.0,20834.0,9612.0,0.398980,100467.400000,163235.989761,27764.0,392459.0,25098.0,367361.0,2.235539,5.0,5,1,09/10/2016,29.942807,0.313382,29.9444,30.7222,29.5000,1.2222,1.002514,40.833341,0.839526,40.5556,42.2222,40.0000,2.2222,0.454498,67.808824,0.758186,68.0,69.0,66.0,3.0,-0.932719,1007.657794,0.230657,1007.720,1007.930,1007.250,0.680,-0.483309,3.526356,1.870559,3.218690,8.04672,0.00000,8.04672,0.260616
3,MA Qing Hua,Marrakesh,2016-2017,Groups Qualifying Session,Q,1.0,TECHEETAH,96.700000,21.620515,89.10,128.5,80.1,48.4,1.761566,3.075050e+04,2.285297e+04,34467.5,54067.0,0.0,54067.0,-0.879319,32754.500000,22510.967172,40873.0,49272.0,0.0,49272.0,-1.666986,17328.750000,12946.936198,19069.5,31176.0,0.0,31176.0,-0.771944,4.0,4,2,12/11/2016,19.256183,0.185109,19.2778,19.5556,18.7778,0.7778,-0.816759,22.608022,1.027137,22.7778,23.8889,20.5556,3.3333,-0.269498,65.138889,0.960737,65.0,68.0,64.0,4.0,0.936207,962.197528,0.152585,962.246,962.415,961.907,0.508,-0.495600,5.096257,2.324215,4.828030,9.65606,0.00000,9.65606,-0.247735
4,MA Qing Hua,Newyork2,2017-2018,Qualifying 2 Group 1,Q,1.0,NIO Formula E Team,74.933333,36.874698,93.10,99.2,32.5,66.7,-1.678880,5.431333e+04,3.739482e+04,33244.0,97489.0,32207.0,65282.0,1.730552,28552.333333,1275.374585,29261.0,29316.0,27080.0,2236.0,-1.728428,27890.333333,1207.169140,27685.0,29187.0,26799.0,2388.0,0.743282,3.0,3,0,07/15/2018,22.518533,0.045397,22.5278,22.5556,22.4444,0.1112,-0.857321,23.333300,0.000000,23.3333,23.3333,23.3333,0.0000,0.000000,89.333333,0.516398,89.0,90.0,89.0,1.0,0.968246,1015.210000,0.000000,1015.210,1015.210,1015.210,0.000,0.000000,3.005982,1.043103,2.652335,4.24374,2.12187,2.12187,0.455943
5,MA Qing Hua,Paris,2017-2018,Qualifying Group 4,Q,4.0,NIO Formula E Team,82.633333,33.691146,93.30,109.7,44.9,64.8,-1.281899,3.527567e+04,2.974910e+04,19071.0,69609.0,17147.0,52462.0,1.723904,27218.666667,2207.172928,27469.0,29290.0,24897.0,4393.0,-0.503816,24313.666667,3295.023874,24447.0,27540.0,20954.0,6586.0,-0.181795,3.0,3,0,04/28/2018,15.333333,0.049685,15.3333,15.3889,15.2778,0.1111,0.001509,21.666700,0.000000,21.6667,21.6667,21.6667,0.0000,0.000000,59.833333,0.408248,60.0,60.0,59.0,1.0,-2.449490,1007.153333,0.025820,1007.150,1007.180,1007.110,0.070,-0.759105,1.768222,0.866251,1.591400,3.1828

In [29]:
final_df = get_encoded_dataframe(final_data_with_weather)
final_df = final_df.fillna(0)
y = final_df['Total_Lap_Num'].to_numpy()
X = final_df.drop(columns = ['Total_Lap_Num','DATE_ONLY']).to_numpy()

In [30]:
final_df.head()

,mean_KPH,std_KPH,med_KPH,max_KPH,min_KPH,range_KPH,skew_KPH,mean_S1,std_S1,med_S1,max_S1,min_S1,range_S1,skew_S1,mean_S2,std_S2,med_S2,max_S2,min_S2,range_S2,skew_S2,mean_S3,std_S3,med_S3,max_S3,min_S3,range_S3,skew_S3,Total_Lap_Num,PIT_TIME_cnt,cnt_CROSSING_FINISH_LINE_IN_PIT,DATE_ONLY,mean_air_temp,std_air_temp,med_air_temp,max_air_temp,min_air_temp,range_air_temp,skew_air_temp,mean_track_temp,std_track_temp,med_track_temp,max_track_temp,min_track_temp,range_track_temp,skew_track_temp,mean_HUMIDITY,std_HUMIDITY,med_HUMIDITY,max_HUMIDITY,min_HUMIDITY,range_HUMIDITY,skew_HUMIDITY,mean_PRESSURE,std_PRESSURE,med_PRESSURE,max_PRESSURE,min_PRESSURE,range_PRESSURE,skew_PRESSURE,mean_WIND_SPEED,std_WIND_SPEED,med_WIND_SPEED,max_WIND_SPEED,min_WIND_SPEED,range_WIND_SPEED,skew_WIND_SPEED,DRIVER_NAME_0,DRIVER_NAME_1,DRIVER_NAME_2,DRIVER_NAME_3,DRIVER_NAME_4,DRIVER_NAME_5,DRIVER_NAME_6,DRIVER_NAME_7,DRIVER_NAME_8,DRIVER_NAME_9,DRIVER_NAME_10,DRIVER_NAME_11,DRIVER_NAME_12,DRIVER_NAME_13,DRIVER_NAME_14,DRIVER_NAME_15,DRIVER_NAME_16,DRIVER_NAME_17,DRIVER_NAME_18,DRIVER_NAME_19,DRIVER_NAME_20,DRIVER_NAME_21,DRIVER_NAME_22,DRIVER_NAME_23,DRIVER_NAME_24,DRIVER_NAME_25,DRIVER_NAME_26,DRIVER_NAME_27,DRIVER_NAME_28,DRIVER_NAME_29,DRIVER_NAME_30,DRIVER_NAME_31,DRIVER_NAME_32,DRIVER_NAME_33,DRIVER_NAME_34,DRIVER_NAME_35,DRIVER_NAME_36,DRIVER_NAME_37,DRIVER_NAME_38,DRIVER_NAME_39,DRIVER_NAME_40,DRIVER_NAME_41,DRIVER_NAME_42,DRIVER_NAME_43,DRIVER_NAME_44,DRIVER_NAME_45,DRIVER_NAME_46,DRIVER_NAME_47,DRIVER_NAME_48,DRIVER_NAME_49,location_0,location_1,location_2,location_3,location_4,location_5,location_6,location_7,location_8,location_9,location_10,location_11,location_12,location_13,location_14,location_15,location_16,location_17,location_18,location_19,season_year_0,season_year_1,match_name_0,match_name_1,match_name_2,match_name_3,match_name_4,match_name_5,match_name_6,match_name_7,match_name_8,match_name_9,match_name_10,match_name_11,match_name_12,match_name_13,match_name_14,match_name_15,match_name_16,match_name_17,match_name_18,match_name_19,match_name_20,match_type_0,match_type_1,match_type_2,GROUP_0,GROUP_1,GROUP_2,GROUP_3,TEAM_0,TEAM_1,TEAM_2,TEAM_3,TEAM_4,TEAM_5,TEAM_6,TEAM_7,TEAM_8,TEAM_9,TEAM_10,TEAM_11,TEAM_12
0,55.950000,74.034080,55.95,108.3,3.6,104.7,0.000000,1.205626e+06,1.673677e+06,1205626.5,2389095.0,22158.0,2366937.0,0.000000,28569.500000,2348.301620,28569.5,30230.0,26909.0,3321.0,0.000000,33244.500000,132.228968,33244.5,33338.0,33151.0,187.0,0.000000,2.0,2,0,02/18/2017,26.299227,0.067998,26.2778,26.4444,26.2222,0.2222,0.634238,26.250000,0.243363,26.1111,26.6667,26.1111,0.5556,1.195861,74.931818,0.899554,75.0,76.0,73.0,3.0,-0.665006,1008.236136,0.126201,1008.230,1008.470,1008.030,0.440,0.053279,4.462272,2.049350,4.828030,8.04672,0.00000,8.04672,0.095071,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.720000,44.706677,83.70,103.4,9.6,93.8,-0.504993,1.454818e+05,2.776263e+05,21419.0,642077.0,18214.0,623863.0,2.235218,25340.400000,3472.268898,25310.0,30446.0,20834.0,9612.0,0.398980,100467.400000,163235.989761,27764.0,392459.0,25098.0,367361.0,2.235539,5.0,5,1,09/10/2016,29.942807,0.313382,29.9444,30.7222,29.5000,1.2222,1.002514,40.833341,0.839526,40.5556,42.2222,40.0000,2.2222,0.454498,67.808824,0.758186,68.0,69.0,66.0,3.0,-0.932719,1007.657794,0.230657,1007.720,1007.930,1007.250,0.680,-0.483309,3.526356,1.870559,3.218690,8.04672,0.00000,8.04672,0.260616,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [33]:
for model in models.keys():
    print('----------------------\n')
    test_model(X,y,model, models[model])
    

----------------------

MODEL: ElasticNet
R2 Value:  0.9977305603891171
MSE Value:  0.08704934116948387
----------------------

MODEL: Random Forest


/opt/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 349.24084804052546, tolerance: 2.1872489642184565
  model = cd_fast.enet_coordinate_descent(


R2 Value:  0.9978424405373133
MSE Value:  0.11064676691729321
----------------------

MODEL: K-Nearest Neighbors
R2 Value:  0.7682084042121882
MSE Value:  12.217142857142857
----------------------

MODEL: SVR
R2 Value:  -0.1402395557541487
MSE Value:  18.719186928285577
----------------------

MODEL: BayesianRidge
R2 Value:  0.9999876460706378
MSE Value:  0.0003979031587047198


In [94]:
X

array([[0.0, 0.6434357121348991, 0.0684797507693635, ..., 0.0, 0.0, 0.0],
       [2.0, 0.1040650406504065, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.6611973392461198, 0.04381984008505998, ..., 0.0, 0.0, 0.0],
       ...,
       [3.0, 0.777638238763938, 0.07313583939478689, ..., 0.0, 0.0, 0.0],
       [4.0, 0.698260625050213, 0.14233153288842068, ..., 0.0, 0.0, 0.0],
       [4.0, 0.7834417932031813, 0.07261645077034812, ..., 0.0, 0.0, 0.0]],
      dtype=object)